In [2]:
# Basics
import numpy as np
import pandas as pd
import random
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Custom
import sys
sys.path.insert(0, '../../src/utils')
from data_loader import DataLoader
import constants
sys.path.insert(0, '../../src/modeling')
from network import Network

# Suppress TensorFlow messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # 1 for INFO, 2 for INFO & WARNINGs, 3 for INFO & WARNINGs & ERRORs

In [3]:
BYTE_IDX = 0
TRAIN_CONFIG = 'D1-K1'
TEST_CONFIGS = ['D1-K6', 'D1-K0', 'D2-K0', 'D3-K0']

# Best HP from Genetic Algorithm (1 device)
# HP = {
#     'first_batch_norm': True,
#     'second_batch_norm': False,
#     'dropout_rate': 0.0,
#     'l1': 0.0,
#     'l2': 0.0,
#     "hidden_layers": 5, 
#     "hidden_neurons": 200, 
#     "optimizer": "adam", 
#     "learning_rate": 0.0001, 
#     "batch_size": 128
# }

# Best hp from genetic algorithm 2 devs, metric=val_loss
HP = {"first_batch_norm": True, "second_batch_norm": True, "hidden_layers": 5, "hidden_neurons": 200, "dropout_rate": 0.1, "l1": 0.0, "l2": 0.001, "optimizer": "adam", "learning_rate": 0.001, "batch_size": 256}

CALLBACKS = [
    EarlyStopping(
        monitor='val_loss', 
        patience=15
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=7,
        min_lr=1e-8
    )
]

## 1 device

In [4]:
print('Loading Train Data...')
train_dl = DataLoader([f'{constants.CURR_DATASETS_PATH}/SBOX_OUT/{TRAIN_CONFIG}_train.json'], BYTE_IDX)
x_train, y_train, _, _ = train_dl.load_data()
print('Loading Val Data...')
val_dl = DataLoader([f'{constants.CURR_DATASETS_PATH}/SBOX_OUT/{TRAIN_CONFIG}_test.json'], BYTE_IDX)
x_val, y_val, _, _ = val_dl.load_data()

print('Loading Test Data...')
x_test_list = []
y_test_list = []
for c in TEST_CONFIGS:
    test_dl = DataLoader([f'{constants.CURR_DATASETS_PATH}/SBOX_OUT/{c}_test.json'], BYTE_IDX)
    x_test, y_test, _, _ = test_dl.load_data()
    x_test_list.append((c, x_test))
    y_test_list.append(y_test)

Loading Train Data...
Loading Val Data...
Loading Test Data...


In [5]:
net = Network('MLP')
net.set_hp(HP)
net.build_model()
model = net.model

model.fit(
    x_train, 
    y_train, 
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=HP['batch_size'],
    callbacks=CALLBACKS,
    verbose=1
)

Epoch 1/100
157/157 [==============================] - 2s 5ms/step - loss: 6.2302 - accuracy: 0.0116 - val_loss: 7.6129 - val_accuracy: 0.0081 - lr: 0.0010
Epoch 2/100
157/157 [==============================] - 0s 3ms/step - loss: 4.9382 - accuracy: 0.0507 - val_loss: 6.7105 - val_accuracy: 0.0287 - lr: 0.0010
Epoch 3/100
157/157 [==============================] - 1s 3ms/step - loss: 4.1961 - accuracy: 0.0818 - val_loss: 4.0986 - val_accuracy: 0.0782 - lr: 0.0010
Epoch 4/100
157/157 [==============================] - 1s 3ms/step - loss: 3.8952 - accuracy: 0.1009 - val_loss: 5.3112 - val_accuracy: 0.0359 - lr: 0.0010
Epoch 5/100
157/157 [==============================] - 1s 3ms/step - loss: 3.6841 - accuracy: 0.1162 - val_loss: 4.9015 - val_accuracy: 0.0572 - lr: 0.0010
Epoch 6/100
157/157 [==============================] - 1s 3ms/step - loss: 3.5500 - accuracy: 0.1241 - val_loss: 3.8346 - val_accuracy: 0.0986 - lr: 0.0010
Epoch 7/100
157/157 [==============================] - 1s 3ms/st

In [6]:
_, val_acc = model.evaluate(x_val, y_val, verbose=0)
print(f'{TRAIN_CONFIG} Val accuracy: {val_acc}')
print()


for i, (c, x) in enumerate(x_test_list):
    y = y_test_list[i]
    _, test_acc = model.evaluate(x, y, verbose=0)
    print(f'{c} Test accuracy (training with {TRAIN_CONFIG}): {test_acc}')
    print()

D1-K1 Val accuracy: 0.2946000099182129

D1-K6 Test accuracy (training with D1-K1): 0.24210000038146973

D1-K0 Test accuracy (training with D1-K1): 0.2013999968767166

D2-K0 Test accuracy (training with D1-K1): 0.0835999995470047

D3-K0 Test accuracy (training with D1-K1): 0.029600000008940697



## 2 devices

In [7]:
TRAIN_CONFIGS = [f'D{d}-K{k}' for k in range(1, 11) for d in [1, 2]]

In [8]:
print('Loading Train Data...')
train_dl = DataLoader([f'{constants.CURR_DATASETS_PATH}/SBOX_OUT/{c}_train.json' for c in TRAIN_CONFIGS], BYTE_IDX)
x_train, y_train, _, _ = train_dl.load_data()
print('Loading Val Data...')
val_dl = DataLoader([f'{constants.CURR_DATASETS_PATH}/SBOX_OUT/{c}_test.json' for c in TRAIN_CONFIGS], BYTE_IDX)
x_val, y_val, _, _ = val_dl.load_data()

print('Loading Test Data...')
x_test_list = []
y_test_list = []
for c in TEST_CONFIGS:
    test_dl = DataLoader([f'{constants.CURR_DATASETS_PATH}/SBOX_OUT/{c}_test.json'], BYTE_IDX)
    x_test, y_test, _, _ = test_dl.load_data()
    x_test_list.append((c, x_test))
    y_test_list.append(y_test)

Loading Train Data...
Loading Val Data...
Loading Test Data...


In [9]:
net2 = Network('MLP')
net2.set_hp(HP)
net2.build_model()
model2 = net2.model

model2.fit(
    x_train, 
    y_train, 
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=HP['batch_size'],
    callbacks=CALLBACKS,
    verbose=1
)

Epoch 1/100
157/157 [==============================] - 1s 5ms/step - loss: 6.4026 - accuracy: 0.0094 - val_loss: 10.8241 - val_accuracy: 0.0036 - lr: 0.0010
Epoch 2/100
157/157 [==============================] - 0s 3ms/step - loss: 4.9903 - accuracy: 0.0482 - val_loss: 8.6740 - val_accuracy: 0.0137 - lr: 0.0010
Epoch 3/100
157/157 [==============================] - 0s 3ms/step - loss: 4.3997 - accuracy: 0.0682 - val_loss: 4.7697 - val_accuracy: 0.0492 - lr: 0.0010
Epoch 4/100
157/157 [==============================] - 0s 3ms/step - loss: 4.1308 - accuracy: 0.0771 - val_loss: 5.3568 - val_accuracy: 0.0327 - lr: 0.0010
Epoch 5/100
157/157 [==============================] - 0s 3ms/step - loss: 3.9604 - accuracy: 0.0841 - val_loss: 4.5679 - val_accuracy: 0.0501 - lr: 0.0010
Epoch 6/100
157/157 [==============================] - 1s 3ms/step - loss: 3.8473 - accuracy: 0.0957 - val_loss: 4.8435 - val_accuracy: 0.0380 - lr: 0.0010
Epoch 7/100
157/157 [==============================] - 0s 3ms/s

In [10]:
_, val_acc = model2.evaluate(x_val, y_val, verbose=0)
print(f'{TRAIN_CONFIGS} Val accuracy: {val_acc}')
print()


for i, (c, x) in enumerate(x_test_list):
    y = y_test_list[i]
    _, test_acc = model2.evaluate(x, y, verbose=0)
    print(f'{c} Test accuracy (training with {TRAIN_CONFIGS}): {test_acc}')
    print()

['D1-K1', 'D2-K1', 'D1-K2', 'D2-K2', 'D1-K3', 'D2-K3', 'D1-K4', 'D2-K4', 'D1-K5', 'D2-K5', 'D1-K6', 'D2-K6', 'D1-K7', 'D2-K7', 'D1-K8', 'D2-K8', 'D1-K9', 'D2-K9', 'D1-K10', 'D2-K10'] Val accuracy: 0.23899999260902405

D1-K6 Test accuracy (training with ['D1-K1', 'D2-K1', 'D1-K2', 'D2-K2', 'D1-K3', 'D2-K3', 'D1-K4', 'D2-K4', 'D1-K5', 'D2-K5', 'D1-K6', 'D2-K6', 'D1-K7', 'D2-K7', 'D1-K8', 'D2-K8', 'D1-K9', 'D2-K9', 'D1-K10', 'D2-K10']): 0.21250000596046448

D1-K0 Test accuracy (training with ['D1-K1', 'D2-K1', 'D1-K2', 'D2-K2', 'D1-K3', 'D2-K3', 'D1-K4', 'D2-K4', 'D1-K5', 'D2-K5', 'D1-K6', 'D2-K6', 'D1-K7', 'D2-K7', 'D1-K8', 'D2-K8', 'D1-K9', 'D2-K9', 'D1-K10', 'D2-K10']): 0.17149999737739563

D2-K0 Test accuracy (training with ['D1-K1', 'D2-K1', 'D1-K2', 'D2-K2', 'D1-K3', 'D2-K3', 'D1-K4', 'D2-K4', 'D1-K5', 'D2-K5', 'D1-K6', 'D2-K6', 'D1-K7', 'D2-K7', 'D1-K8', 'D2-K8', 'D1-K9', 'D2-K9', 'D1-K10', 'D2-K10']): 0.16259999573230743

D3-K0 Test accuracy (training with ['D1-K1', 'D2-K1', 'D1-K